<a href="https://colab.research.google.com/github/dvschultz/ai/blob/master/Ganspace_S2DD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#GANSPACE - Discovering Interpretable GAN Controls 

Using https://github.com/harskish/ganspace to find latent directions in a stylegan2 model. (This could easily be ported to other models, if anyone implements it please get in touch, and i'll add it to the notebook!)

Notebook put together by [@realmeatyhuman](https://twitter.com/realmeatyhuman), simplified and edited by Derrick Schultz for the Artificial Images StyleGAN Deep Dive course.



In [0]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [0]:
# title Mount Google Drive
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)


## Setup
We need to install some dependencies. This takes a little longer than usual compared to other repos. The second cell especially can take a while.


In [0]:
# Clone git
!git clone https://github.com/harskish/ganspace
%cd ganspace
!pip install fbpca pyopengltk glumpy

In [0]:
#@title Install remaining packages
!git submodule update --init --recursive
!python -c "import nltk; nltk.download('wordnet')"
!wget https://files.pythonhosted.org/packages/5e/3f/5658c38579b41866ba21ee1b5020b8225cec86fe717e4b1c5c972de0a33c/pycuda-2019.1.2.tar.gz
!tar -xzf pycuda-2019.1.2.tar.gz
%cd /content/ganspace/pycuda-2019.1.2
!python configure.py --cuda-enable-gl --cuda-root=/usr/local/cuda
!make install
%cd "/content/ganspace"
%cd models/stylegan2/stylegan2-pytorch/op
!python -c "import torch; import upfirdn2d_op; import fused; print('OK')"
!python setup.py install
%cd "/content/ganspace"

# Convert model weights
(skip this step if you already have a pytorch model)

Because GANspace is written in PyTorch (rather than Tensorflow, which is what we’ve used for StyleGAN2 previously) we need to convert our .pkl model file to a PyTorch format.


In [0]:
#download your model
!gdown --id 1UlDmJVLLnBD9SnLSMXeiZRO6g-OMQCA_ -O /content/ffhq.pkl

In [0]:
#we need the tensorflow model to build the model network, then we convert it to the pytorch model file.
%cd "/content"
!git clone https://github.com/skyflynil/stylegan2
%cd ganspace

The convert weight script takes two arguments: 

```
--repo - Path to tensorflow stylegan2 repo
       - Path to your model
```



In [0]:
!python /content/ganspace/models/stylegan2/stylegan2-pytorch/convert_weight.py --repo="/content/stylegan2/" "/content/floralmag.pkl" #convert weights

In [0]:
#this can move your newly created model file (.pt) to Google Drive if you'd like
!cp "/content/stylegan2/floralmag.pt" "/content/drive/My Drive/ganspace" #copy pytorch model to your drive

# Run PCA Analysis

We’re now ready to run the PCA script. To do this we’ll need to open GANspace and edit one of the files to include our custom model.

From here, open models/wrappers.py (in Colab you can just double-click on the file), and edit the StyleGAN2 configs dict on line 110 to include your model and its corresponding resolution.

I.E from

        # Image widths
        configs = {
            # Converted NVIDIA official
            'ffhq': 1024,
            'car': 512,
            'cat': 256,
            'church': 256,
            'horse': 256,
            # Tuomas
            'bedrooms': 256,
            'kitchen': 256,
            'places': 256,
        }

to 

        # Image widths
        configs = {
            # Converted NVIDIA official
            'your_model': your_resolution,
            'ffhq': 1024,
            'car': 512,
            'cat': 256,
            'church': 256,
            'horse': 256,
            # Tuomas
            'bedrooms': 256,
            'kitchen': 256,
            'places': 256,
        }

Then copy your pytorch model over to your drive account or any other hosting platform, and add the direct download link to the checkpoints dict in the download_checkpoint function on line 136.

    def download_checkpoint(self, outfile):
        checkpoints = { 
            'yourmodel': 'https://drive.google.com/yourmodel',
            'horse': 'https://drive.google.com/uc?export=download&id=18SkqWAkgt0fIwDEf2pqeaenNi4OoCo-0',
            'ffhq': 'https://drive.google.com/uc?id=12yYXZymadSIj74Yue1Q7RrlbIqrXggo3',
            'church': 'https://drive.google.com/uc?export=download&id=1HFM694112b_im01JT7wop0faftw9ty5g',
            'car': 'https://drive.google.com/uc?export=download&id=1iRoWclWVbDBAy5iXYZrQnKYSbZUqXI6y',
            'cat': 'https://drive.google.com/uc?export=download&id=15vJP8GDr0FlRYpE8gD7CdeEz2mXrQMgN',
            'places': 'https://drive.google.com/uc?export=download&id=1X8-wIH3aYKjgDZt4KMOtQzN1m4AlCVhm',
            'bedrooms': 'https://drive.google.com/uc?export=download&id=1nZTW7mjazs-qPhkmbsOLLA_6qws-eNQu',
            'kitchen': 'https://drive.google.com/uc?export=download&id=15dCpnZ1YLAnETAPB0FGmXwdBclbwMEkZ'
        }




#Set PCA Options

Next let’s tell GANspace what model we want to use and how many components we want to generate.

```
Command line paramaters:
  --model      one of [ProGAN, BigGAN-512, BigGAN-256, BigGAN-128, StyleGAN, StyleGAN2]
  --class      class name; leave empty to list options
  --layer      layer at which to perform PCA; leave empty to list options
  --use_w      treat W as the main latent space (StyleGAN / StyleGAN2)
  --inputs     load previously exported edits from directory
  --sigma      number of stdevs to use in visualize.py
  -n           number of PCA samples
  -b           override automatic minibatch size detection
  -c           number of components to keep

```




In [0]:
%cd ../ganspace/

/content/ganspace


In [0]:
# set base variables
model = 'StyleGAN2' 
model_class = 'ladiescrop' #this is the name of your model in the configs
num_components = 20

In [0]:
#Check layers available for analysis
!python visualize.py --model $model --class $model_class --use_w
!python interactive.py --model=StyleGAN2 --class=car --layer=style -n=1000000 -b=10000

In [0]:
!python visualize.py --model=$model --class=$model_class --use_w --layer="style" -c $num_components -n=1000000 -b=10000 --batch --video  #add --video to generate videos

In [0]:
#zip up your samples for download
!zip -r samples-c80-ladiescrop2.zip "/content/ganspace/out/StyleGAN2-ladiescrop" #zip up samples for download

In [0]:
#copy your samples to Google Drive
%cp -r "/content/ganspace/samples-c80-ladiescrop.zip" "/content/drive/My Drive/ganspace" #copying components over to google drive

# Explore Directions!

After running visualize.py, your components will be stored in an npz file in /content/ganspace/cache/components/ - below the npz file is unpacked, and a component/direction is chosen at random. 

Using the UI, you can explore the latent direction and give it a name, which will be appeneded to the named_directions dictionary and saved as 'direction_name.npy' for later use.

From here, you may want to copy the components over to your drive.


In [0]:
path_to_tf_stylegan = '/content/stylegan2'
path_to_model = '/content/ladiescrop-network-snapshot-011308.pkl'
path_to_components = '/content/ganspace/cache/components/stylegan2-ladiescrop_style_ipca_c80_n300000_w.npz'
named_directions = {} #init named_directions dict to save directions

In [0]:
#@title Load model { display-mode: "both" }

#@markdown This code loads the tensorflow version of StyleGAN2
%cd $path_to_tf_stylegan
import tensorflow as tf
import ipywidgets as widgets
import pretrained_networks
import PIL.Image
import numpy as np
import argparse
import numpy as np
import PIL.Image
import dnnlib
import dnnlib.tflib as tflib
import re
import sys
from io import BytesIO
import IPython.display
import numpy as np
from math import ceil
from PIL import Image, ImageDraw
import imageio
import pretrained_networks

src_model = path_to_model

_G, _D, Gs = pretrained_networks.load_networks(src_model)
Gs_syn_kwargs = dnnlib.EasyDict()
batch_size = 1
Gs_syn_kwargs.output_transform = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
Gs_syn_kwargs.randomize_noise = True
Gs_syn_kwargs.minibatch_size = batch_size

noise_vars = [var for name, var in Gs.components.synthesis.vars.items() if name.startswith('noise')]

/content/stylegan2


##Load a component vector at random

This code will pick one of the components at random (by default there are 80 components, but you can set this when running the PCA code above)



In [0]:
#component_value = 12
component_value = 'random'

comps = np.load(path_to_components)
lst = comps.files
latent_dirs = []
latent_stdevs = []

load_activations = True

for item in lst:
    # loading latent directions
    # TODO: what is the difference btwn act_comp and latent_comp? should I use one, the other or both?

    # print('\n')
    # print('--------'*10)
    # print(item)
    if load_activations:
      if item == 'act_comp':
        # print(comps[item].shape[0])
        for i in range(comps[item].shape[0]):
          # print(comps[item][i].shape)
          latent_dirs.append(comps[item][i])
      if item == 'act_stdev':
        # print()
        for i in range(comps[item].shape[0]):
          latent_stdevs.append(comps[item][i])
    else:
      if item == 'lat_comp':
        # print(comps[item].shape[0])
        for i in range(comps[item].shape[0]):
          # print(comps[item][i].shape)
          latent_dirs.append(comps[item][i])
      if item == 'lat_stdev':
        # print()
        for i in range(comps[item].shape[0]):
          latent_stdevs.append(comps[item][i])
        
    # print('--------'*10)
    # print('\n')
    

#load one at random 
if component_value == 'random':
  num = np.random.randint(num_components)
else:
  num = component_value

if num in named_directions.values():
  print(f'Direction already named: {list(named_directions.keys())[list(named_directions.values()).index(num)]}')

random_dir = latent_dirs[num]
random_dir_stdev = latent_dirs[num]

random_dir = np.expand_dims(random_dir, axis=0)
#TODO: I think currently this will only work for 1024x1024 models? make this adjustable but idk the rule
# for 512 x 512 models it's 16, 1024 x 1024 = 18
n = 18
if Gs.input_shape[1:][0] == 512:
  n = 18
random_dir = np.tile(random_dir, [1, n, 1]) 

print(f'Loaded Component No. {num}')


Loaded Component No. 12


In [0]:
#@title Run UI
from IPython.utils import io

def name_direction(sender):
  if num in named_directions.values():
    target_key = list(named_directions.keys())[list(named_directions.values()).index(num)]
    print(f'Direction already named: {target_key}')
    print(f'Overwriting... ')
    del(named_directions[target_key])
  named_directions[text.value] = num
  save_direction(random_dir, text.value)
  for item in named_directions:
    print(item, named_directions[item])

def save_direction(direction, filename):
  filename += ".npy"
  np.save(filename, direction, allow_pickle=True, fix_imports=True)
  print(f'Latent direction saved as {filename}')

# Taken from https://github.com/alexanderkuk/log-progress
def log_progress(sequence, every=1, size=None, name='Items'):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{name}: {index} / ?'.format(
                        name=name,
                        index=index
                    )
                else:
                    progress.value = index
                    label.value = u'{name}: {index} / {size}'.format(
                        name=name,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = "{name}: {index}".format(
            name=name,
            index=str(index or '?')
        )

def generate_mov(seed, truncation, direction_vec, scale, n_frames, out_name = 'out', noise_spec = None, loop=True):
  """Generates a mov moving back and forth along the chosen direction vector"""
  # Example of reading a generated set of images, and storing as MP4.
  %mkdir out
  movieName = f'out/{out_name}.mp4'
  offset = -10
  step = 20 / n_frames
  imgs = []
  for i in range(n_frames):
    print(f'{i} / {n_frames}')
    Gs_kwargs = dnnlib.EasyDict()
    Gs_kwargs.output_transform = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
    Gs_kwargs.randomize_noise = False
    if truncation is not None:
        Gs_kwargs.truncation_psi = truncation
            
    rnd = np.random.RandomState(seed)
        
    if noise_spec is None:
      tflib.set_vars({var: rnd.randn(*var.shape.as_list()) for var in noise_vars}) # [height, width]
    else:
      tflib.set_vars(noise_spec)
    
    batch_size = 1
    all_seeds = [seed] * batch_size
    all_z = np.stack([np.random.RandomState(seed).randn(*Gs.input_shape[1:]) for seed in all_seeds]) # [minibatch, component]
    all_w = Gs.components.mapping.run(all_z, None) # [minibatch, layer, component]
    if truncation != 1:
        w_avg = Gs.get_var('dlatent_avg')
        all_w = w_avg + (all_w - w_avg) * truncation # [minibatch, layer, component]
    all_w += direction_vec * offset * scale
    all_images = Gs.components.synthesis.run(all_w, **Gs_syn_kwargs)
    #save image and display
    final_im = PIL.Image.fromarray(np.median(all_images, axis=0).astype(np.uint8))
    imgs.append(final_im)
    #increase offset
    offset += step
  if loop:
    imgs += imgs[::-1]
  with imageio.get_writer(movieName, mode='I') as writer:
    for image in log_progress(list(imgs), name = "Creating animation"):
        writer.append_data(np.array(image))

def display_sample(seed, truncation, distance, scale, disp=True, save=None, noise_spec=None):
    # blockPrint()
    with io.capture_output() as captured:
      # weighted_average(Gs, Gsd, blending)
      
      Gs_kwargs = dnnlib.EasyDict()
      Gs_kwargs.output_transform = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
      Gs_kwargs.randomize_noise = False
      if truncation is not None:
          Gs_kwargs.truncation_psi = truncation
          
      rnd = np.random.RandomState(seed)
      
      if noise_spec is None:
        tflib.set_vars({var: rnd.randn(*var.shape.as_list()) for var in noise_vars}) # [height, width]
      else:
        tflib.set_vars(noise_spec)
      
      batch_size = 1
      all_seeds = [seed] * batch_size
      all_z = np.stack([np.random.RandomState(seed).randn(*Gs.input_shape[1:]) for seed in all_seeds]) # [minibatch, component]
      all_w = Gs.components.mapping.run(all_z, None) # [minibatch, layer, component]
      if truncation != 1:
          w_avg = Gs.get_var('dlatent_avg')
          all_w = w_avg + (all_w - w_avg) * truncation # [minibatch, layer, component]
      print(random_dir)
      print(distance)
      print(scale)
      all_w += random_dir * distance * scale
      all_images = Gs.components.synthesis.run(all_w, **Gs_syn_kwargs)
      #save image and display
      final_im = PIL.Image.fromarray(np.median(all_images, axis=0).astype(np.uint8)).resize((400,400),Image.LANCZOS)
    if disp:
      # print('no display')
      display(final_im)
    if save is not None:
      if disp == False:
        print(save)
      final_im.save(f'out/{seed}_{save:05}.png')

seed = np.random.randint(0,100000)
style = {'description_width': 'initial'}

seed = widgets.IntSlider(min=0, max=100000, step=1, value=seed, description='Seed: ', continuous_update=False)
truncation = widgets.FloatSlider(min=0, max=2, step=0.1, value=0.7, description='Truncation: ', continuous_update=False)
distance = widgets.FloatSlider(min=-10, max=10, step=0.1, value=0, description='Distance: ', continuous_update=False, style=style)
scale = widgets.FloatSlider(min=0, max=5, step=0.05, value=1, description='Scale: ', continuous_update=False)
text = widgets.Text(description="Name component here", style=style, width=200)

bot_box = widgets.HBox([seed, truncation, distance, scale, text])
ui = widgets.VBox([bot_box])

out = widgets.interactive_output(display_sample, {'seed': seed, 'truncation': truncation, 'distance': distance, 'scale': scale})

display(ui, out)
text.on_submit(name_direction)




Output()

## Generate interpolations from named vectors

In [0]:
#script to generate a movie moving back and forth along the direction

direction_name = 'shift-color'
loc = named_directions[direction_name]
num_samples = 20

for i in range(num_samples):
  s = np.random.randint(0, 10000)
  generate_mov(seed = s, truncation = 0.8, direction_vec = latent_dirs[loc], scale = 1, n_frames = 50, out_name = f'{model_class}_{direction_name}_{i}', loop=True)

In [0]:
#title Select from named directions

from IPython.display import display, clear_output

vardict = list(named_directions.keys())
select_variable = widgets.Dropdown(
    options=vardict,
    value=vardict[0],
    description='Select variable:',
    disabled=False,
    button_style=''
)

def set_direction(b):
    clear_output()
    print(select_variable.value)
    random_dir = latent_dirs[named_directions[select_variable.value]]
    seed = np.random.randint(0,100000)
    style = {'description_width': 'initial'}

    seed = widgets.IntSlider(min=0, max=100000, step=1, value=seed, description='Seed: ', continuous_update=False)
    truncation = widgets.FloatSlider(min=-2, max=2, step=0.1, value=0.7, description='Truncation: ', continuous_update=False)
    distance = widgets.FloatSlider(min=-10, max=10, step=0.1, value=0, description='Distance: ', continuous_update=False, style=style)
    scale = widgets.FloatSlider(min=0, max=5, step=0.05, value=1, description='Scale: ', continuous_update=False)

    bot_box = widgets.HBox([seed, truncation, distance, scale])
    ui = widgets.VBox([bot_box])

    out = widgets.interactive_output(display_sample, {'seed': seed, 'truncation': truncation, 'distance': distance, 'scale': scale})
    display(select_variable)
    display(ui, out)

random_dir = latent_dirs[named_directions[select_variable.value]]
seed = np.random.randint(0,100000)
style = {'description_width': 'initial'}

seed = widgets.IntSlider(min=0, max=100000, step=1, value=seed, description='Seed: ', continuous_update=False)
truncation = widgets.FloatSlider(min=-2, max=2, step=0.1, value=0.7, description='Truncation: ', continuous_update=False)
distance = widgets.FloatSlider(min=-10, max=10, step=0.1, value=0, description='Distance: ', continuous_update=False, style=style)
scale = widgets.FloatSlider(min=0, max=5, step=0.05, value=1, description='Scale: ', continuous_update=False)

bot_box = widgets.HBox([seed, truncation, distance, scale])
ui = widgets.VBox([bot_box])

out = widgets.interactive_output(display_sample, {'seed': seed, 'truncation': truncation, 'distance': distance, 'scale': scale})

display(select_variable)

select_variable.observe(set_direction, names='value')



Dropdown(description='Select variable:', options=('shift-color',), value='shift-color')

# Download Vectors
You can download your named vectors for use outside of this Colab.

In the StyleGAN2 folder your vectors will be named `{name}.npy`.